In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler

X_train = pd.read_csv("dataset/train_features.csv")
y_train = pd.read_csv("dataset/train_labels.csv")
X_test = pd.read_csv("dataset/valid_features.csv")
y_test = pd.read_csv("dataset/valid_labels.csv")

X_train = pd.DataFrame(X_train, columns=["loudness","title","tempo", "time_signature","key","mode","duration"])
X_test = pd.DataFrame(X_test, columns=["loudness","title","tempo", "time_signature","key","mode","duration"])

cleanup_nums = {"genre": 
                {"classic pop and rock": 1, 
                 "dance and electronica": 2,
                 "folk":3,
                 "jazz and blues":4,
                 "metal":5,
                 "pop":6,
                 "punk":7,
                 "soul and reggae":8
                }}
y_train.replace(cleanup_nums, inplace=True)
y_test.replace(cleanup_nums, inplace=True)


tfidf_vect = TfidfVectorizer()
X_title_cv_train = tfidf_vect.fit_transform(X_train['title'])
df_title_cv_train = pd.DataFrame(X_title_cv_train.toarray(), columns=tfidf_vect.get_feature_names())
X_train = pd.concat([X_train, df_title_cv_train], axis=1)
X_train = X_train.drop(["title"], 1)

X_title_cv_test = tfidf_vect.transform(X_test['title'])
df_title_cv_test = pd.DataFrame(X_title_cv_test.toarray(), columns=tfidf_vect.get_feature_names())
X_test = pd.concat([X_test, df_title_cv_test], axis=1)
X_test = X_test.drop(["title"], 1)

scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)


In [2]:
lgr = LogisticRegression(random_state=0)
lgr.fit(X_train,y_train["genre"])
lgr_pred = lgr.predict(X_test)

# let's how our model performed
print("Classificaion report: \n")
print(classification_report(y_test["genre"], lgr_pred))
print("Confusion matrix: \n")
print(confusion_matrix(y_test["genre"], lgr_pred))

Classificaion report: 

              precision    recall  f1-score   support

           1       0.24      0.51      0.32        55
           2       0.20      0.02      0.04        45
           3       0.31      0.53      0.40        64
           4       1.00      0.02      0.04        44
           5       0.66      0.73      0.69        66
           6       0.87      0.53      0.66        74
           7       0.43      0.59      0.50        44
           8       0.38      0.26      0.31        58

    accuracy                           0.43       450
   macro avg       0.51      0.40      0.37       450
weighted avg       0.52      0.43      0.40       450

Confusion matrix: 

[[28  0 18  0  1  0  3  5]
 [13  1 10  0  6  1  7  7]
 [19  1 34  0  4  0  3  3]
 [18  1 15  1  3  0  0  6]
 [ 7  0  1  0 48  0 10  0]
 [14  0 11  0  1 39  7  2]
 [ 3  1  7  0  2  4 26  1]
 [16  1 12  0  8  1  5 15]]


/home/wildan/.local/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
